<a href="https://colab.research.google.com/github/Nivratti/100-Days-Of-ML-Code/blob/master/stylegan_encoder_generating_image_dataset_for_pix2pixHD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Dec 19 15:35:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!git clone https://github.com/Nivratti/stylegan2encoder.git

Cloning into 'stylegan2encoder'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 316 (delta 3), reused 5 (delta 1), pack-reused 308
Receiving objects: 100% (316/316), 8.99 MiB | 4.15 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [14]:
%cd "/content/stylegan2encoder"

/content/stylegan2encoder


## Generate images

In [20]:
## Info
# # truncation-psi -- affectes face generation
# #  1) truncation-psi above 0.5 creates more dark and little red colored faces and below 0.5 creates yello and white faces
# #  2) truncation-psi near 1 generates eye glasses  and truncation-psi near 0 reduces glass level.

# !python run_generator.py generate-images --network=gdrive:networks/stylegan2-ffhq-config-f.pkl \
#   --seeds=6600-6625 --truncation-psi=0.5

In [ ]:
# !python run_generator.py generate-images-custom --network=gdrive:networks/stylegan2-ffhq-config-f.pkl \
#     --truncation-psi=0.7 \
#     --num 50 \
#     --result-dir generated_ffhq_smile  \
#     --direction_path latent_directions/smile.npy \
#     --coeff 1.5

In [19]:
## latent directions info
## manipulate face
## 1) default encoder latents -- 
## 2) downloaded from twitter -- stored in tw folder

!python run_generator.py generate-images-custom --network=gdrive:networks/stylegan2-ffhq-config-f.pkl \
    --truncation-psi=0.5 \
    --num 50 \
    --result-dir generated_ffhq_age  \
    --direction_path latent_directions/tw/age.npy \
    --coeff 6.5

Local submit - run_dir: generated_ffhq_age/00000-generate-images-custom
dnnlib: Running run_generator.generate_images_custom() on localhost...
Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
100%|##########| 6/6 [01:25<00:00, 14.18s/it]
dnnlib: Finished run_generator.generate_images_custom() in 1m 35s.


### Transform data in pix2pixHD format

In [ ]:
import shutil
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
generated_dataset_path = Path('generated_ffhq_smile/images')
new_dataset_path = Path('ffhq_smile_pix2pixHD')

In [ ]:
train_A_path = new_dataset_path / 'train_A'
train_B_path = new_dataset_path / 'train_B'
train_C_path = new_dataset_path / 'train_C'

train_A_path.mkdir(parents=True, exist_ok=True)
train_B_path.mkdir(parents=True, exist_ok=True)
train_C_path.mkdir(parents=True, exist_ok=True)

In [ ]:
all_paths = list(generated_dataset_path.iterdir())

In [ ]:
source_paths = [p for p in all_paths if 'tr' not in str(p)]
neg_paths = [p for p in all_paths if 'tr' in str(p) and '-' in str(p)]
pos_paths = [p for p in all_paths if 'tr' in str(p) and '-' not in str(p)]

In [ ]:
len(source_paths), len(neg_paths), len(pos_paths)

(544, 544, 544)

In [ ]:
for i, (source, neg, pos) in tqdm(enumerate(zip(sorted(source_paths, key=lambda x: x.name.split('.')[0] + '_'),
                                                sorted(neg_paths), sorted(pos_paths)))):
    shutil.copy(source, train_A_path / f'{i}.png')
    shutil.copy(pos, train_B_path / f'{i}.png')
    shutil.copy(neg, train_C_path / f'{i}.png')

544it [00:13, 39.46it/s]
